In [ ]:
#!pip install tensorflow keras numpy pillow h5py

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
#!wget http://images.cocodataset.org/zips/train2014.zip
#!wget http://images.cocodataset.org/zips/val2014.zip
#!wget http://images.cocodataset.org/annotations/annotations_trainval2014.zip

#!mkdir -p ./data/images
#!unzip train2014.zip -d ./data/images/
#!unzip val2014.zip -d ./data/images/
#!unzip annotations_trainval2014.zip -d ./data/

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!rm -r ~/.kaggle
! mkdir ~/.kaggle

In [ ]:
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#! kaggle datasets list

In [ ]:
! kaggle datasets download stanford-Image-Paragraph-Captioning-dataset

403 - Forbidden


In [ ]:
#!wget http://cs.stanford.edu/people/rak248/VG_100K_2/images.zip
#!wget http://cs.stanford.edu/people/rak248/VG_100K_2/images2.zip
#!wget http://cs.stanford.edu/people/rak248/VG_100K_2/paragraphs_v1.json.zip


In [ ]:
import json
import numpy as np
import os
import re
import string
import random
import tensorflow as tf
import urllib.request
from PIL import Image
#from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils import pad_sequences
#from keras.utils.text import Tokenizer
from keras.preprocessing.text import Tokenizer

In [ ]:
# Load the COCO dataset
def load_coco_dataset1(images_path, annotations_path, max_images=None):
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    image_paths = []
    captions = []
    for annotation in annotations['annotations']:
        image_id = annotation['image_id']
        caption = annotation['caption']
        path = os.path.join(images_path, 'COCO_train2014_' + str(image_id).zfill(12) + '.jpg')
        if not os.path.exists(path):
            continue
        image_paths.append(path)
        captions.append(caption)
    if max_images is not None:
        image_paths = image_paths[:max_images]
        captions = captions[:max_images]
    return image_paths, captions

# Load the COCO dataset
def load_coco_dataset2(images_path, annotations_path, max_images=None):
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    image_paths = []
    captions = []
    image_ids = set()
    for annotation in annotations['annotations']:
        image_id = annotation['image_id']
        if image_id in image_ids:
            continue
        image_ids.add(image_id)
        caption = annotation['caption']
        path = os.path.join(images_path, 'COCO_train2014_' + str(image_id).zfill(12) + '.jpg')
        if not os.path.exists(path):
            continue
        image_paths.append(path)
        captions.append(caption)
    if max_images is not None:
        image_paths = image_paths[:max_images]
        captions = captions[:max_images]
    return image_paths, captions

def load_coco_dataset(images_path, annotations_path, max_images=None):
    with open(annotations_path, 'r') as f:
        annotations = json.load(f)
    image_paths = []
    captions = []
    for annotation in annotations['annotations']:
        image_id = annotation['image_id']
        caption = annotation['caption']
        path = os.path.join(images_path, 'COCO_train2014_' + str(image_id).zfill(12) + '.jpg')
        if not os.path.exists(path):
            continue
        image_paths.append(path)
        captions.append(caption)
    if max_images is not None:
        # Randomly select a subset of images
        indices = random.sample(range(len(image_paths)), max_images)
        image_paths = [image_paths[i] for i in indices]
        captions = [captions[i] for i in indices]
    return image_paths, captions
# Preprocess the images
def preprocess_images(image_paths):
    # Load the image using Pillow
    def load_image(path):
        img = Image.open(path)
        img = img.convert('RGB')
        img = img.resize((299, 299))  # Xception model input size
        img = np.array(img) / 255.0
        img = np.expand_dims(img, axis=0)
        return img

    images = []
    for path in image_paths:
        images.append(load_image(path))
    images = np.vstack(images)
    return images

# Create a tokenizer for the captions
def create_tokenizer(captions, num_words=None):
    tokenizer = Tokenizer(num_words=num_words, oov_token="<unk>", filters=string.punctuation)
    tokenizer.fit_on_texts(captions)
    return tokenizer

# Create a vocabulary of words in the captions
def create_vocabulary(captions):
    vocab = set()
    for caption in captions:
        tokens = caption.split()
        vocab.update(tokens)
    return sorted(list(vocab))

# Create input-output sequences from the captions
def create_sequences(tokenizer, captions, max_length):
    sequences = tokenizer.texts_to_sequences(captions)
    padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')
    input_sequences = padded_sequences[:, :-1]
    output_sequences = padded_sequences[:, 1:]
    return input_sequences, output_sequences

# Generate a paragraph describing the contents of an image
def generate_image_paragraph(image_path, model, tokenizer, max_length):
    # Load and preprocess the image
    img = preprocess_images([image_path])[0]
    # Extract visual features using the Xception model
    visual_features = model.predict(img)
    # Generate captions using the image captioning model
    input_caption = np.zeros((1, max_length))
    input_caption[0, 0] = tokenizer.word_index['<start>']



In [ ]:
#images_path = './data/images/train2014/'
#annotations_path = './data/annotations/captions_train2014.json'

# Load the COCO dataset
#image_paths, captions = load_coco_dataset(images_path, annotations_path, max_images=11000)

#print(len(image_paths))
#print(captions)

Preprocess the data

In [ ]:
images = preprocess_images(image_paths)

In [ ]:
from keras.applications.xception import Xception
from keras.applications.xception import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.models import Model
#model = Xception(weights='imagenet', include_top=False)
model=InceptionV3(weights='imagenet')
model= Model(model.input, model.layers[-2].output)
# summarize
print(model.summary())



# Preprocess images for Xception model
#x = preprocess_input(images)

# Extract visual features
#visual_features = model.predict(x)


Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_11 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 block1_conv1 (Conv2D)          (None, None, None,   864         ['input_11[0][0]']               
                                32)                                                               
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, None, None,   128        ['block1_conv1[0][0]']           
 ation)                         32)                                                        

Prepare language data

In [ ]:

#captions = [ann["caption"] for ann in subset_data["annotations"].values()]

# Create tokenizer and vocabulary
tokenizer = create_tokenizer(captions)
vocab_size = 5000
vocab = create_vocabulary(captions)
sequences, max_length = create_sequences(tokenizer, captions, vocab_size)


AttributeError: ignored

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

# Model parameters
embedding_size = 300
lstm_units = 256
dropout_rate = 0.5

# Define input layers
image_input = Input(shape=(2048,))
caption_input = Input(shape=(max_length,))

# Define embedding layer
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_size, mask_zero=True)(caption_input)

# Define LSTM layers
lstm1 = LSTM(lstm_units)(embedding)
dropout1 = Dropout(dropout_rate)(lstm1)
lstm2 = LSTM(lstm_units)(dropout1)
dropout2 = Dropout(dropout_rate)(lstm2)

# Define output layer
output = Dense(vocab_size, activation="softmax")(dropout2)

# Define the model
model = Model(inputs=[image_input, caption_input], outputs=output)
model.summary()

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, concatenate

# Model parameters
embedding_size = 300
word_lstm_units = 512
sentence_lstm_units = 512
dropout_rate = 0.5

# Define input layers
visual_input = Input(shape=(4096,))
caption_input = Input(shape=(40,))

# Define embedding layer
embedding = Embedding(input_dim=12114, output_dim=embedding_size, mask_zero=True)(caption_input)

# Define word-level LSTM layer
word_lstm = LSTM(word_lstm_units, return_sequences=True)(embedding)
word_dropout = Dropout(dropout_rate)(word_lstm)

# Define sentence-level LSTM layer
sentence_lstm = LSTM(sentence_lstm_units)(word_dropout)
sentence_dropout = Dropout(dropout_rate)(sentence_lstm)

# Concatenate image features and sentence LSTM output
concatenated = concatenate([visual_input, sentence_dropout])

# Define output layer
output = Dense(12114, activation='softmax')(concatenated)

# Define the model
model = Model(inputs=[visual_input, caption_input], outputs=output)
model.summary()


In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, concatenate, RepeatVector, TimeDistributed
from keras.applications.xception import Xception
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.xception import preprocess_input
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D

# Model parameters
embedding_size = 300
word_lstm_units = 512
sentence_lstm_units = 512
dropout_rate = 0.5

# Define input layers
image_input = Input(shape=(299, 299, 3))
caption_input = Input(shape=(40,))

# Define Xception model
xception_model = Xception(include_top=False, pooling='avg')
visual_input = xception_model(image_input)

# Define region detector layers
region_conv = Conv2D(512, (3, 3), padding='same', activation='relu')(xception_model.output)
region_pool = MaxPooling2D(pool_size=(17, 17))(region_conv)
region_dropout = Dropout(dropout_rate)(region_pool)
region_flat = TimeDistributed(Flatten())(region_dropout)

# Define embedding layer
embedding = Embedding(input_dim=12114, output_dim=embedding_size, mask_zero=True)(caption_input)

# Define word-level LSTM layer
word_lstm = LSTM(word_lstm_units, return_sequences=True)(embedding)
word_dropout = Dropout(dropout_rate)(word_lstm)

# Define sentence-level LSTM layer
sentence_lstm = LSTM(sentence_lstm_units)(word_dropout)
sentence_dropout = Dropout(dropout_rate)(sentence_lstm)

# Repeat sentence-level LSTM output for each region
sentence_repeat = RepeatVector(196)(sentence_dropout)

# Concatenate region features and repeated sentence LSTM output
concatenated = concatenate([region_flat, sentence_repeat])

# Define output layer
output = TimeDistributed(Dense(12114, activation='softmax'))(concatenated)

# Define the model
model = Model(inputs=[image_input, caption_input], outputs=output)
model.summary()


In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout, concatenate, TimeDistributed
import tensorflow as tf

# Model parameters
vocab_size = 12114
embedding_size = 300
word_lstm_units = 512
sentence_lstm_units = 512
pooling_size = 2048
topic_vector_size = 256
dropout_rate = 0.5

# Define input layers
image_input = Input(shape=(224, 224, 3))
region_input = Input(shape=(None, 7, 7, 2048))
caption_input = Input(shape=(None,))

# Define embedding layer
embedding = Embedding(input_dim=vocab_size, output_dim=embedding_size, mask_zero=True)(caption_input)

# Define region pooling layer
region_max = tf.math.reduce_max(region_input, axis=(2, 3))  # max-pool over spatial dimensions
region_max = tf.keras.layers.Reshape((tf.shape(region_max)[1], pooling_size))(region_max)  # reshape to match sentence lstm input

# Define sentence-level LSTM layer
sentence_lstm = LSTM(sentence_lstm_units, return_sequences=True)(region_max)
sentence_dropout = Dropout(dropout_rate)(sentence_lstm)

# Define sentence-level topic vector generation
topic_vector = TimeDistributed(Dense(topic_vector_size, activation='tanh'))(sentence_dropout)

# Define word-level LSTM layer
word_lstm = LSTM(word_lstm_units, return_sequences=True)(embedding)
word_dropout = Dropout(dropout_rate)(word_lstm)

# Define word-level topic vector concatenation
word_concat = concatenate([word_dropout, topic_vector], axis=2)

# Define output layer
output = TimeDistributed(Dense(vocab_size, activation='softmax'))(word_concat)

# Define the model
model = Model(inputs=[image_input, region_input, caption_input], outputs=output)
model.summary()


In [ ]:
from keras.optimizers import Adam

learning_rate = 0.001

optimizer = Adam(lr=learning_rate)
model.compile(loss="categorical_crossentropy", optimizer=optimizer)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Create an image generator
datagen = ImageDataGenerator()
generator = datagen.flow(
    visual_features,
    sequences,
    batch_size=64,
)

# Train the model
epochs = 50
steps_per_epoch = len(subset_data["annotations"]) // 64

model.fit(
    generator,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    verbose=1,
)


In [ ]:
from generate import generate_image_paragraph

# Load a new image
new_image_path = "path/to/new/image.jpg"
new_image = preprocess_image(new_image_path, image_size)

# Extract visual features
new_visual_features = model.predict(new_image)

# Generate paragraph
paragraph = generate_image_paragraph(model, new_visual_features, tokenizer, max_length, vocab)
print(paragraph)
